In [ ]:
import tensorflow as tf

In [ ]:
file=open("Word.txt","r",encoding = "utf8")
lines=[]
for i in file:
  lines.append(i)
#conversion of list to string



In [ ]:
data=""
for i in lines:
  data=' '.join(lines)


In [ ]:
data=data.replace('\n','').replace('"','').replace("\ufeff",'').replace('***','').replace('”','').replace('“','')
data=data.split()
data=' '.join(data)
data[:500]



"heart's content; every one was very kind, and she had three compliments. Annie made her sing, and some one said she had a remarkably fine voice; Major Lincoln asked who the fresh little girl, with the beautiful eyes, was; and Mr. Moffat insisted on dancing with her, because she didn't dawdle, but had some spring in her, as he gracefully expressed it. So, altogether, she had a very nice time, till she overheard a bit of a conversation, which disturbed her extremely. She was sitting just inside th"

**Tokenizer**

In [ ]:
from keras.preprocessing.text import Tokenizer
import pickle
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer,open('aa.pkl','wb'))
sequence=tokenizer.texts_to_sequences([data])[0]

sequence[:15]


[1837, 1308, 128, 32, 13, 50, 224, 1, 10, 25, 319, 1838, 1605, 92, 7]

In [ ]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

8713

In [ ]:
import numpy as np
sequences=[]
n=3
for i in range(3,len(sequence)):
  words=sequence[i-n:i+1]
  sequences.append(words)
sequences=np.array(sequences)

In [ ]:
X=[]
Y=[]

for i in sequences:
  X.append(i[0:n])
  Y.append(i[n])
X=np.array(X)
Y=np.array(Y)

In [ ]:
print("Inputs:",X[:15])
print("Outputs:",Y[:15])


Inputs: [[1837 1308  128]
 [1308  128   32]
 [ 128   32   13]
 [  32   13   50]
 [  13   50  224]
 [  50  224    1]
 [ 224    1   10]
 [   1   10   25]
 [  10   25  319]
 [  25  319 1838]
 [ 319 1838 1605]
 [1838 1605   92]
 [1605   92    7]
 [  92    7 1309]
 [   7 1309    1]]
Outputs: [  32   13   50  224    1   10   25  319 1838 1605   92    7 1309    1
   93]


In [ ]:
from keras.utils import to_categorical
Y=to_categorical(Y,num_classes=vocab_size,dtype="float32")
Y[:5]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
model=Sequential()
model.add(Embedding(vocab_size,10,input_length=n))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000,activation="relu"))
model.add(Dense(vocab_size,activation="softmax"))



In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             87130     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8713)              8721713   
                                                                 
Total params: 21,857,843
Trainable params: 21,857,843
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
checkpoint=ModelCheckpoint("next_word.h5",monitor='loss',verbose=1,save_best_only=True)
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(X,Y,epochs=20,batch_size=64,callbacks=[checkpoint])


Epoch 1/20
1757/1758 [============================>.] - ETA: 0s - loss: 6.5370 - accuracy: 0.0541
Epoch 1: loss improved from inf to 6.53679, saving model to next_word.h5
1758/1758 [==============================] - 38s 18ms/step - loss: 6.5368 - accuracy: 0.0541
Epoch 2/20
1757/1758 [============================>.] - ETA: 0s - loss: 6.0189 - accuracy: 0.0876
Epoch 2: loss improved from 6.53679 to 6.01879, saving model to next_word.h5
1758/1758 [==============================] - 31s 18ms/step - loss: 6.0188 - accuracy: 0.0876
Epoch 3/20
1757/1758 [============================>.] - ETA: 0s - loss: 5.7018 - accuracy: 0.1029
Epoch 3: loss improved from 6.01879 to 5.70170, saving model to next_word.h5
1758/1758 [==============================] - 31s 17ms/step - loss: 5.7017 - accuracy: 0.1029
Epoch 4/20
1756/1758 [============================>.] - ETA: 0s - loss: 5.4501 - accuracy: 0.1146
Epoch 4: loss improved from 5.70170 to 5.45020, saving model to next_word.h5
1758/1758 [==============

In [ ]:
model.evaluate(X,Y)

3516/3516 [==============================] - 21s 6ms/step - loss: 1.2052


1.205177903175354

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_word.h5')
tokenizer = pickle.load(open('aa.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word


In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

['i', 'am', 'a']
1/1 [==============================] - 1s 805ms/step
stupid


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-10cbba31caae>", line 2, in <cell line: 2>
    text = input("Enter your line: ")
  File "/usr/local/lib/python3.9/dist-packages/ipykernel/kernelbase.py", line 851, in raw_input
    return self._input_request(str(prompt),
  File "/usr/local/lib/python3.9/dist-packages/ipykernel/kernelbase.py", line 895, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the ab

TypeError: ignored